In [11]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch

In [12]:
data = pd.read_csv('dataset/RAW_recipes.csv')
#print(data.head())
sample_size = int(len(data) * 0.001)
data_sample = data.sample(n=sample_size)
ingredients = data_sample['ingredients']
instructions = data_sample['steps']
print(ingredients.head())
print(instructions.head())

156467    ['spaghetti', 'broccoli florets', 'chunky pean...
147896    ['olive oil', 'spicy sausage', 'onion', 'carro...
133179    ['halibut', 'fresh lemon juice', 'olive oil', ...
125385    ['water', 'sugar', 'orange, zest of', 'lemons,...
9287      ['salmon fillets', 'dijon mustard', 'soy sauce...
Name: ingredients, dtype: object
156467    ['bring a large pot water to boil', 'cook past...
147896    ['heat oil in a skillet over medium heat and f...
133179    ['mix halibut , lemon juice and 4 teaspoons oi...
125385    ['bring water and sugar to boil and stir to di...
9287      ['heat grill and brush rack with olive oil to ...
Name: steps, dtype: object


In [13]:
# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Prepare the data for training
inputs = tokenizer(ingredients.tolist(), return_tensors='pt', truncation=True, padding=True)
labels = torch.tensor([1]*len(ingredients))  # assuming all recipes are "positive"

In [15]:
# Define the optimizer
optimizer = Adam(model.parameters(), lr=1e-5) 

In [16]:
# Fine-tune the model
model.train()
num_epochs = 1  # Define the number of epochs
batch_size = 32  # Define the batch size

for epoch in range(num_epochs):
    for i, batch in enumerate(DataLoader(list(zip(inputs['input_ids'], inputs['attention_mask'], labels)), batch_size=batch_size)):
        optimizer.zero_grad()
        input_ids, attention_mask, label_batch = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [1]:
input_ids = tokenizer.encode('chicken, rice, carrots', return_tensors='pt')
generated = model.generate(input_ids, max_length=100, do_sample=True)
recipe = tokenizer.decode(generated[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

print(recipe)

NameError: name 'tokenizer' is not defined